#       **1. PRE-PROCESAMIENTO DE IMAGENES ALOS PALSAR Y VARIABLES EXPLICATIVAS**

Permite el analisis de imagenes satelitales en la nube utilizando Google Earth Engine; permitiendo obtener una imagen  Alos Palsar con índices explicativos para el modelo para el área de interés, de acuerdo con su retrodispersión. 
Ayuda a obtener información estructural de la vegetación a la que se le medirá la AGBD

## 1.1 Importar librerias y cargar área de interés

In [1]:
import contextily as ctx
import ee
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely
from shapely.geometry import Point

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project="ee-coberturap")

Enter verification code:  4/1AVMBsJjCFpTieqjGTC-8Dy6Gf4-ataUtKxOs39hH67RTEmPptPxNLhe4KK4



Successfully saved authorization token.


In [3]:
# --- File paths  ---
root_folder = r"/notebooks/ProyectoFinal/Shapes/"
adi_route = root_folder + "/PNNAmacayacu.shp"
adi = gpd.read_file(adi_route).to_crs(epsg=9377)
adi

,objectid_1,nombre,categoria,territoria,resolucion,hectareas_,escala,organizaci,fecha_act,url,...,centroid_y,fecha_regi,validado,hectarea_1,perimetro_,administra,id_pnn,record_id,app_id,geometry
0,198761,Amacayacu,Parque Nacional Natural,Territorial Amazonia,283,267480.32,None,Parques Nacionales Naturales de Colombia,2011-06-30,https://runap.parquesnacionales.gov.co/area-pr...,...,-3.439523,2011-06-30,Si,267240.603115,None,PNNC,27,2069,1,"POLYGON ((5331866.854 1221528.042, 5331899.919..."


## 1.2 Convertir geometria local a formato Earth Engine


In [4]:
ee_fc = geemap.gdf_to_ee(
    adi
)  # Esto convierte todo el shapefile a una EE FeatureCollection
adi = ee_fc.geometry()

## 1.3 Llamar la colección ScanSAR de Alos Pasar y seleccionar las bandas de interés

In [5]:
dataset = (
    ee.ImageCollection("JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR")
    .filterDate("2023-12-01", "2023-12-31")
    .filterBounds(adi)
)
# Mostrar info sobre la colección
info = dataset.getInfo()
print("Número de imágenes:", len(info["features"]))

# Seleccionar la banda HH, hacer un mosaico y recortar al área
sar_hh = dataset.select("HH").mosaic().clip(adi)

# Seleccionar la banda HH, hacer un mosaico y recortar al área
sar_hv = dataset.select("HV").mosaic().clip(adi)

Número de imágenes: 5


## 1.4 Crear un mapa interactivo centrado en el área de estudio

In [13]:
Map = geemap.Map()
Map.centerObject(adi)
sar_rgb = sar_hh.addBands(sar_hv)

sar_rgb_vis = {"min": 0, "max": 10000, "bands": ["HH", "HV", "HH"]}

Map.addLayer(sar_rgb, sar_rgb_vis, "SAR_RGB", True)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## 1.5 Extracción de variables explicativas como índices y bandas de interes para el modelo

In [7]:
##Se define el path donde están almacenadas las funciones para llamarlas posteriormente

import sys
sys.path.append("/notebooks/ProyectoFinal/src")

In [8]:
import importlib
##las funciones se importan como librerías
import SARFunction
from SARFunction import dn_to_gamma_db, db_to_gamma_pw


In [9]:
##se aplican las funciones

sar_db = dn_to_gamma_db(sar_rgb)
sar_pw = db_to_gamma_pw(sar_db)

print(sar_pw.bandNames().getInfo())

['HH_pw', 'HV_pw']


In [15]:
##Mapa interactivo de los índices calculados

Map2 = geemap.Map()
Map2.centerObject(adi)

sar_pw_vis = {"min": 0, "max": 0.5, "bands": ["HH_pw", "HV_pw", "HH_pw"]}

Map2.addLayer(sar_pw, sar_pw_vis, "SAR_PW", True)
Map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## 1.6. Exportar la composción de SAR

In [11]:
# Export the composite image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=sar_pw,
    description='AlosPalsar_PW',
    folder='earthengine',            # Destination folder in Google Drive
    fileNamePrefix='SAR_PW', # File name without extension
    region=adi,                   # Area to export
    ##scale=25,                        # Set a very high limit to allow exporting large images
                                     # Earth Engine requires this to avoid exporting by mistake very large rasters
                                     # The default limit is 1e8 (100 million pixels), so we override it here

    maxPixels=1e13,                  # Allow large exports
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

print("Export task started. Monitor progress in the Earth Engine Tasks tab.")


Export task started. Monitor progress in the Earth Engine Tasks tab.
